In [1]:
import xml.etree.ElementTree as ET
from pandas import Series, DataFrame
import os
import sys
import argparse
import cv2

classes = ['basketball_shooting','biking','diving','golf_swing','horse_riding','soccer_juggling','swing','tennis_swing','trampoline_jumping','volleyball_spiking','walking','do_nothing']

In [2]:
dataSetFolder = '/Users/thoithanh/Downloads/Dataset/UCF11_updated_mpg/'
pathOutObj = '/Users/thoithanh/Desktop/obj/'
pathOutTxt = '/Users/thoithanh/Desktop/obj/'
interval = 10
errorfiles = ['v_tennis_09_01.xgtf','v_tennis_09_02.xgtf','v_tennis_14_04.xgtf','v_biking_08_06.xgtf']

### Number of classes (12 classes)
0 - basketball_shooting
1 - biking
2 - diving
3 - golf_swing
4 - horse_riding
5 - soccer_juggling
6 - swing
7 - tennis_swing
8 - trampoline_jumping
9 - volleyball_spiking
10 - walking
11 - do_nothing

In [3]:
def get_position(att):
    fr, to = (int(x) for x in att['framespan'].split(':') )
    h = int(att['height'])
    w = int(att['width'])
    x = int(att['x'])
    y = int(att['y'])
    return fr,to,h,w,x,y

In [4]:
def gen_df(obj,interval,pathOut,fileName):
    #inverval when action occurs
    ac_interval = 2
    
    frameno, h, w, x, y = [], [], [], [], []
    c = []
    
    #tete = (int(x) for x in obj.attrib['framespan'].split(':'))
    tete = obj.attrib['framespan'].split(':')
    objfrom = int(tete[0])
    objto = int(tete[-1])
    cursor = objfrom
    al = interval # 10 frames
    for iframe in obj[0]:
        efr,eto,eh,ew,ex,ey = get_position(iframe.attrib)
        while(cursor >= efr and cursor <= eto):
            frameno.append(cursor)
            h.append(eh)
            w.append(ew)
            x.append(ex)
            y.append(ey)
            do_loop = True
            c_appended = False
            for atts in obj[1:]:
                if not do_loop:
                    break
                for bvalue in atts:
                    if (bvalue.attrib['value']=='true'):
                        ifr, ito = (int(x) for x in bvalue.attrib['framespan'].split(':'))
                        if cursor in range(ifr,ito+1):
                            c.append(atts.attrib['name'])
                            c_appended = True
                            #cursor -= (al-ac_interval)
                            do_loop = False
                            break
            if not c_appended:
                c.append('do_nothing')      
            cursor += al

    df1 = DataFrame({'FrameNo':frameno,'Height':h,'Width':w,'X':x,'Y':y,'Class':c})
    #print(df1)
    
    
    for idx,i in enumerate(frameno):
        x_center = 0
        y_center = 0
        re_w = w[idx]
        re_h = h[idx]
        
        file = open(pathOut+fileName+'_'+str(i)+'.txt','a')
        if(os.path.getsize(pathOut+fileName+'_'+str(i)+'.txt') > 0):
            file.write('\n'+str(classes.index(c[idx]))+' '+str(x_center)+' '+str(y_center)+' '+str(w[idx])+' '+str(h[idx]))
        else:
            file.write(str(classes.index(c[idx]))+' '+str(x_center)+' '+str(y_center)+' '+str(w[idx])+' '+str(h[idx]))
        file.close()

In [5]:
# [frameno, h, w, x, y, c] = [[]]*6

In [6]:
# xmlfile = ET.parse('v_shooting_01_01.xml')
# root = xmlfile.getroot()
# data = root[1][1] #last sourcefile info

In [7]:
def gen_df2(obj,interval,ac_interval):
    [frameno, h, w, x, y, c] = [[]]*6
    
    for idx, obj in enumerate(data[1:]):
        #tete = (int(x) for x in obj.attrib['framespan'].split(':'))
        tete = obj.attrib['framespan'].split(':')
        objfrom = int(tete[0])
        objto = int(tete[-1])
        


In [8]:
def gen_txt(pathIn,pathOut,interval,fileName):
    xmlfile = ET.parse(pathIn)
    root = xmlfile.getroot()
    data = root[1][0] #last sourcefile info 
    
    for idx, obj in enumerate(data[1:]):
        #tete = (int(x) for x in obj.attrib['framespan'].split(':'))
        tete = obj.attrib['framespan'].split(':')
        objfrom = int(tete[0])
        objto = int(tete[-1])
        #print("Object no:",str(idx))
        #print("Frame to - from:",objfrom,objto)
        gen_df(obj,interval,pathOut,fileName)
        #print()

In [9]:
# pathIn = 'v_shooting_01_01.xml'
# #fileName = 'v_shooting_01_01'

# gen_txt(pathIn,'/Users/thoithanh/Desktop/txtfolder/',10)

In [10]:
def extractImages(pathIn, pathOut, interval):
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    count = 1
    success = True
    i = 1
    while success:
        success,image = vidcap.read()
        if i != count:
            i += 1
            continue
        #print ('Read a new frame: ', success)
        cv2.imwrite( pathOut + '_' + "%d.jpg" % count, image)     # save frame as JPEG file
        print('Write %s_%d successfully' % (pathOut, count))
        count += interval
        i += 1

In [11]:
def dataSet2Img():
    for classFolder in os.listdir(dataSetFolder):
        path_classFolder = dataSetFolder+classFolder+'/'
        if(os.path.isdir(path_classFolder)):
            for folders in os.listdir(path_classFolder):
                if folders == 'Annotation':
                    continue
                path_folders = path_classFolder+folders+'/'
                if(os.path.isdir(path_folders)):
                    for vid in os.listdir(path_folders):
                        path_vid = path_folders+vid+'/'
                        vidName = vid[:-4]
                        extractImages(path_vid,pathOutObj+vidName,interval)

In [12]:
def anno2txt():    
    for classFolder in os.listdir(dataSetFolder):
        path_classFolder = dataSetFolder+classFolder+'/'
        if(os.path.isdir(path_classFolder)):
            for folders in os.listdir(path_classFolder):
                if folders == 'Annotation':
                    path_ann = path_classFolder+folders+'/'
                    for files in os.listdir(path_ann):
                        if files[-3:] == 'txt':
                            continue
                        if files in errorfiles:
                            continue
                        print(files)
                        path_files = path_ann+files
                        fileName = files[:-5]
                        gen_txt(path_files,pathOutTxt,interval,fileName)
                        #Check 

                    break

In [13]:
dataSet2Img()
anno2txt()

Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_31 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_41 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_51 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_61 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_71 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_81 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_91 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_101 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_111 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_121 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_04_131 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_12_01_1 succes

Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_121 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_131 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_141 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_151 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_161 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_171 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_181 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_01_191 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_31 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_41 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_51 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_23_03_61 s

Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_101 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_111 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_121 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_131 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_141 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_151 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_161 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_03_171 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_31 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_41 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_51 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_25_01_61 s

Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_31 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_41 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_51 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_61 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_71 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_81 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_91 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_06_101 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_07_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_07_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_07_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_13_07_31 successfu

Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_31 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_41 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_51 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_06_61 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_1 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_11 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_21 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_31 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_41 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_51 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_04_61 successfully
Write /Users/thoithanh/Desktop/obj/v_spiking_07_05_1 successfull

...

Write /Users/thoithanh/Desktop/obj/v_golf_05_02_81 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_02_91 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_1 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_11 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_21 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_31 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_41 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_51 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_61 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_71 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_81 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_91 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_101 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_111 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_05_08_121 successfully
Write /Users/thoithanh/Desktop/obj/v_g

Write /Users/thoithanh/Desktop/obj/v_golf_04_01_71 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_04_01_81 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_04_01_91 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_04_01_101 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_04_01_111 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_04_01_121 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_1 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_11 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_21 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_31 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_41 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_51 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_61 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_71 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_03_04_81 successfully
Write /Users/thoithanh/Desktop/obj/v_g

Write /Users/thoithanh/Desktop/obj/v_golf_10_04_101 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_04_111 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_1 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_11 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_21 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_31 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_41 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_51 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_61 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_71 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_81 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_91 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_01_101 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_03_1 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_10_03_11 successfully
Write /Users/thoithanh/Desktop/obj/v_go

Write /Users/thoithanh/Desktop/obj/v_golf_17_02_91 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_17_02_101 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_17_02_111 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_17_02_121 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_17_08_1 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_17_08_11 successfully
Write /Users/thoithanh/Desktop/obj/v_golf_17_08_21 successfully


KeyboardInterrupt: 

In [24]:
from PIL import Image
def ifilter():
    for img in os.listdir(pathOutObj):
        if img[-3:] == 'jpg':
            pathImg = pathOutObj + img
            try:
                imgss = Image.open(pathImg)
            except Exception:
                os.remove(pathImg)
                print('Invalid image: '+img)
    
    for img in os.listdir(pathOutObj):
        if img[-3:] == 'jpg':
            pathImg = pathOutObj + img
            txtFileName = pathImg[:-3] + 'txt'
            print(txtFileName)
            if not os.path.isfile(txtFileName):
                print("Remove " + pathImg)
                os.remove(pathImg)
    
    for txt in os.listdir(pathOutObj):
        if txt[-3:] == 'txt':
            pathTxt = pathOutObj + txt
            imgFileName = pathTxt[:-3] + 'jpg'
            if not os.path.isfile(imgFileName):
                print("Remove " + pathTxt)
                os.remove(pathTxt)

In [26]:
#ifilter()

/Users/thoithanh/Desktop/obj/v_riding_19_01_111.txt
/Users/thoithanh/Desktop/obj/v_golf_15_02_1.txt
/Users/thoithanh/Desktop/obj/v_biking_04_03_41.txt
/Users/thoithanh/Desktop/obj/v_jumping_11_04_31.txt
/Users/thoithanh/Desktop/obj/v_golf_02_04_1.txt
/Users/thoithanh/Desktop/obj/v_spiking_20_04_11.txt
/Users/thoithanh/Desktop/obj/v_jumping_02_01_71.txt
/Users/thoithanh/Desktop/obj/v_juggle_17_04_151.txt
/Users/thoithanh/Desktop/obj/v_riding_25_01_131.txt
/Users/thoithanh/Desktop/obj/v_juggle_17_06_281.txt
/Users/thoithanh/Desktop/obj/v_golf_14_02_111.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_06_03_191.txt
/Users/thoithanh/Desktop/obj/v_tennis_22_03_1.txt
/Users/thoithanh/Desktop/obj/v_swing_16_05_131.txt
/Users/thoithanh/Desktop/obj/v_biking_21_01_1.txt
/Users/thoithanh/Desktop/obj/v_swing_02_03_211.txt
/Users/thoithanh/Desktop/obj/v_shooting_19_08_41.txt
/Users/thoithanh/Desktop/obj/v_shooting_19_05_121.txt
/Users/thoithanh/Desktop/obj/v_swing_23_04_11.txt
/Users/thoithanh/Desktop/o

/Users/thoithanh/Desktop/obj/v_golf_18_05_301.txt
/Users/thoithanh/Desktop/obj/v_spiking_01_04_11.txt
/Users/thoithanh/Desktop/obj/v_tennis_07_07_121.txt
/Users/thoithanh/Desktop/obj/v_jumping_23_01_71.txt
/Users/thoithanh/Desktop/obj/v_tennis_10_04_1.txt
/Users/thoithanh/Desktop/obj/v_biking_02_06_171.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_15_03_41.txt
/Users/thoithanh/Desktop/obj/v_biking_16_02_181.txt
/Users/thoithanh/Desktop/obj/v_juggle_15_03_21.txt
/Users/thoithanh/Desktop/obj/v_biking_25_03_41.txt
/Users/thoithanh/Desktop/obj/v_spiking_07_05_81.txt
/Users/thoithanh/Desktop/obj/v_spiking_12_01_51.txt
/Users/thoithanh/Desktop/obj/v_swing_14_04_91.txt
/Users/thoithanh/Desktop/obj/v_tennis_07_02_1.txt
/Users/thoithanh/Desktop/obj/v_juggle_14_09_141.txt
/Users/thoithanh/Desktop/obj/v_riding_19_08_61.txt
/Users/thoithanh/Desktop/obj/v_shooting_25_03_1.txt
/Users/thoithanh/Desktop/obj/v_tennis_11_03_71.txt
/Users/thoithanh/Desktop/obj/v_diving_03_07_101.txt
/Users/thoithanh/Deskto

/Users/thoithanh/Desktop/obj/v_biking_24_05_111.txt
/Users/thoithanh/Desktop/obj/v_tennis_07_01_21.txt
/Users/thoithanh/Desktop/obj/v_jumping_18_01_51.txt
/Users/thoithanh/Desktop/obj/v_tennis_24_06_121.txt
/Users/thoithanh/Desktop/obj/v_riding_04_02_51.txt
/Users/thoithanh/Desktop/obj/v_riding_11_06_81.txt
/Users/thoithanh/Desktop/obj/v_tennis_04_05_71.txt
/Users/thoithanh/Desktop/obj/v_biking_10_06_221.txt
/Users/thoithanh/Desktop/obj/v_tennis_18_06_101.txt
/Users/thoithanh/Desktop/obj/v_biking_21_07_171.txt
/Users/thoithanh/Desktop/obj/v_tennis_13_06_121.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_15_01_91.txt
/Users/thoithanh/Desktop/obj/v_spiking_07_07_51.txt
/Users/thoithanh/Desktop/obj/v_spiking_12_03_81.txt
/Users/thoithanh/Desktop/obj/v_swing_01_02_91.txt
/Users/thoithanh/Desktop/obj/v_biking_25_01_91.txt
/Users/thoithanh/Desktop/obj/v_golf_17_08_221.txt
/Users/thoithanh/Desktop/obj/v_biking_02_03_181.txt
/Users/thoithanh/Desktop/obj/v_golf_03_03_21.txt
/Users/thoithanh/Deskto

/Users/thoithanh/Desktop/obj/v_diving_19_01_171.txt
/Users/thoithanh/Desktop/obj/v_spiking_04_01_11.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_05_02_91.txt
/Users/thoithanh/Desktop/obj/v_spiking_17_04_51.txt
/Users/thoithanh/Desktop/obj/v_swing_11_01_91.txt
/Users/thoithanh/Desktop/obj/v_swing_04_05_41.txt
/Users/thoithanh/Desktop/obj/v_biking_20_06_41.txt
/Users/thoithanh/Desktop/obj/v_tennis_16_01_171.txt
/Users/thoithanh/Desktop/obj/v_shooting_16_06_1.txt
/Users/thoithanh/Desktop/obj/v_riding_19_01_91.txt
/Users/thoithanh/Desktop/obj/v_tennis_05_02_171.txt
/Users/thoithanh/Desktop/obj/v_diving_08_03_81.txt
/Users/thoithanh/Desktop/obj/v_tennis_11_06_181.txt
/Users/thoithanh/Desktop/obj/v_jumping_10_06_41.txt
/Users/thoithanh/Desktop/obj/v_jumping_05_02_91.txt
/Users/thoithanh/Desktop/obj/v_swing_21_02_31.txt
/Users/thoithanh/Desktop/obj/v_tennis_20_05_101.txt
/Users/thoithanh/Desktop/obj/v_juggle_20_05_81.txt
/Users/thoithanh/Desktop/obj/v_golf_25_02_11.txt
/Users/thoithanh/Desktop

/Users/thoithanh/Desktop/obj/v_riding_04_08_21.txt
/Users/thoithanh/Desktop/obj/v_juggle_16_04_311.txt
/Users/thoithanh/Desktop/obj/v_diving_20_05_61.txt
/Users/thoithanh/Desktop/obj/v_golf_15_04_1.txt
/Users/thoithanh/Desktop/obj/v_golf_02_02_1.txt
/Users/thoithanh/Desktop/obj/v_jumping_03_02_51.txt
/Users/thoithanh/Desktop/obj/v_jumping_16_06_81.txt
/Users/thoithanh/Desktop/obj/v_biking_16_05_21.txt
/Users/thoithanh/Desktop/obj/v_biking_21_06_171.txt
/Users/thoithanh/Desktop/obj/v_biking_04_01_101.txt
/Users/thoithanh/Desktop/obj/v_juggle_21_04_311.txt
/Users/thoithanh/Desktop/obj/v_golf_25_03_41.txt
/Users/thoithanh/Desktop/obj/v_swing_21_03_61.txt
/Users/thoithanh/Desktop/obj/v_tennis_22_05_1.txt
/Users/thoithanh/Desktop/obj/v_juggle_18_06_351.txt
/Users/thoithanh/Desktop/obj/v_spiking_22_03_61.txt
/Users/thoithanh/Desktop/obj/v_tennis_21_05_141.txt
/Users/thoithanh/Desktop/obj/v_diving_12_05_161.txt
/Users/thoithanh/Desktop/obj/v_biking_15_01_71.txt
/Users/thoithanh/Desktop/obj/v_

...

/Users/thoithanh/Desktop/obj/v_biking_20_08_231.txt
/Users/thoithanh/Desktop/obj/v_diving_01_04_11.txt
/Users/thoithanh/Desktop/obj/v_riding_18_05_181.txt
/Users/thoithanh/Desktop/obj/v_shooting_22_03_61.txt
/Users/thoithanh/Desktop/obj/v_juggle_19_08_81.txt
/Users/thoithanh/Desktop/obj/v_golf_18_05_121.txt
/Users/thoithanh/Desktop/obj/v_tennis_13_05_71.txt
/Users/thoithanh/Desktop/obj/v_biking_09_03_61.txt
/Users/thoithanh/Desktop/obj/v_juggle_16_02_211.txt
/Users/thoithanh/Desktop/obj/v_riding_06_06_81.txt
/Users/thoithanh/Desktop/obj/v_riding_13_02_51.txt
/Users/thoithanh/Desktop/obj/v_juggle_02_04_131.txt
/Users/thoithanh/Desktop/obj/v_riding_25_21_141.txt
/Users/thoithanh/Desktop/obj/v_riding_15_06_121.txt
/Users/thoithanh/Desktop/obj/v_shooting_01_05_81.txt
/Users/thoithanh/Desktop/obj/v_shooting_14_01_51.txt
/Users/thoithanh/Desktop/obj/v_riding_06_05_121.txt
/Users/thoithanh/Desktop/obj/v_spiking_18_03_151.txt
/Users/thoithanh/Desktop/obj/v_jumping_01_04_101.txt
/Users/thoithan

/Users/thoithanh/Desktop/obj/v_juggle_14_04_51.txt
/Users/thoithanh/Desktop/obj/v_tennis_16_03_131.txt
/Users/thoithanh/Desktop/obj/v_spiking_13_06_21.txt
/Users/thoithanh/Desktop/obj/v_diving_25_03_111.txt
/Users/thoithanh/Desktop/obj/v_juggle_15_03_381.txt
/Users/thoithanh/Desktop/obj/v_biking_24_02_161.txt
/Users/thoithanh/Desktop/obj/v_biking_15_03_231.txt
/Users/thoithanh/Desktop/obj/v_riding_03_06_61.txt
/Users/thoithanh/Desktop/obj/v_diving_12_04_71.txt
/Users/thoithanh/Desktop/obj/v_shooting_21_02_11.txt
/Users/thoithanh/Desktop/obj/v_tennis_16_05_91.txt
/Users/thoithanh/Desktop/obj/v_tennis_03_01_41.txt
/Users/thoithanh/Desktop/obj/v_tennis_10_02_261.txt
/Users/thoithanh/Desktop/obj/v_biking_18_02_141.txt
/Users/thoithanh/Desktop/obj/v_riding_10_03_21.txt
/Users/thoithanh/Desktop/obj/v_diving_12_03_111.txt
/Users/thoithanh/Desktop/obj/v_diving_01_01_31.txt
/Users/thoithanh/Desktop/obj/v_biking_10_03_181.txt
/Users/thoithanh/Desktop/obj/v_juggle_21_02_361.txt
/Users/thoithanh/D

/Users/thoithanh/Desktop/obj/v_biking_07_06_231.txt
/Users/thoithanh/Desktop/obj/v_spiking_04_01_21.txt
/Users/thoithanh/Desktop/obj/v_diving_19_01_141.txt
/Users/thoithanh/Desktop/obj/v_biking_22_03_191.txt
/Users/thoithanh/Desktop/obj/v_juggle_16_07_81.txt
/Users/thoithanh/Desktop/obj/v_juggle_03_03_51.txt
/Users/thoithanh/Desktop/obj/v_biking_20_06_71.txt
/Users/thoithanh/Desktop/obj/v_tennis_16_01_141.txt
/Users/thoithanh/Desktop/obj/v_golf_15_01_81.txt
/Users/thoithanh/Desktop/obj/v_swing_04_05_71.txt
/Users/thoithanh/Desktop/obj/v_spiking_17_04_61.txt
/Users/thoithanh/Desktop/obj/v_riding_03_04_71.txt
/Users/thoithanh/Desktop/obj/v_diving_12_06_61.txt
/Users/thoithanh/Desktop/obj/v_tennis_10_06_11.txt
/Users/thoithanh/Desktop/obj/v_tennis_21_06_101.txt
/Users/thoithanh/Desktop/obj/v_juggle_18_05_311.txt
/Users/thoithanh/Desktop/obj/v_diving_12_06_121.txt
/Users/thoithanh/Desktop/obj/v_riding_10_01_31.txt
/Users/thoithanh/Desktop/obj/v_diving_01_03_21.txt
/Users/thoithanh/Desktop/

/Users/thoithanh/Desktop/obj/v_jumping_20_02_31.txt
/Users/thoithanh/Desktop/obj/v_swing_11_06_41.txt
/Users/thoithanh/Desktop/obj/v_swing_04_02_91.txt
/Users/thoithanh/Desktop/obj/v_spiking_17_03_81.txt
/Users/thoithanh/Desktop/obj/v_biking_20_01_91.txt
/Users/thoithanh/Desktop/obj/v_juggle_05_05_21.txt
/Users/thoithanh/Desktop/obj/v_swing_20_06_131.txt
/Users/thoithanh/Desktop/obj/v_juggle_18_05_111.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_05_05_41.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_10_01_91.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_10_05_181.txt
/Users/thoithanh/Desktop/obj/v_golf_02_04_101.txt
/Users/thoithanh/Desktop/obj/v_golf_06_03_21.txt
/Users/thoithanh/Desktop/obj/v_juggle_15_04_261.txt
/Users/thoithanh/Desktop/obj/v_riding_25_16_141.txt
/Users/thoithanh/Desktop/obj/v_spiking_11_02_11.txt
/Users/thoithanh/Desktop/obj/v_juggle_01_02_141.txt
/Users/thoithanh/Desktop/obj/v_spiking_08_06_121.txt
/Users/thoithanh/Desktop/obj/v_jumping_11_01_171.txt
/Users/thoithanh/D

/Users/thoithanh/Desktop/obj/v_jumping_23_02_101.txt
/Users/thoithanh/Desktop/obj/v_jumping_16_03_51.txt
/Users/thoithanh/Desktop/obj/v_shooting_04_02_101.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_22_04_221.txt
/Users/thoithanh/Desktop/obj/v_swing_08_01_1.txt
/Users/thoithanh/Desktop/obj/v_jumping_01_02_181.txt
/Users/thoithanh/Desktop/obj/v_juggle_16_03_31.txt
/Users/thoithanh/Desktop/obj/v_riding_12_07_151.txt
/Users/thoithanh/Desktop/obj/v_shooting_17_01_101.txt
/Users/thoithanh/Desktop/obj/v_spiking_11_01_41.txt
/Users/thoithanh/Desktop/obj/v_spiking_04_05_91.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_25_01_101.txt
/Users/thoithanh/Desktop/obj/v_swing_17_04_81.txt
/Users/thoithanh/Desktop/obj/v_juggle_20_02_131.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_14_02_181.txt
/Users/thoithanh/Desktop/obj/v_walk_dog_16_03_51.txt
/Users/thoithanh/Desktop/obj/v_juggle_16_02_571.txt
/Users/thoithanh/Desktop/obj/v_juggle_11_03_261.txt
/Users/thoithanh/Desktop/obj/v_swing_18_01_111.txt
/Users

In [28]:
def gen_traintxt():
    st = './build/darknet/x64/data/obj/'
    for img in os.listdir(pathOutObj):
        if img[-3:] == 'jpg':
            file = open('/Users/thoithanh/Desktop/train.txt','a')
            if(os.path.getsize('/Users/thoithanh/Desktop/train.txt') > 0):
                file.write('\n'+st+img)
            else:
                file.write(st+img)
            file.close()
gen_traintxt()